<a href="https://colab.research.google.com/github/maelmitc/ProjetIA-Ma-l-Manon/blob/main/Project/coucou%20manon%20hehe/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [4]:
from google.colab import files

uploaded = files.upload()

Saving 1d-monopoly-regle.pdf to 1d-monopoly-regle.pdf


In [5]:
file_name = list(uploaded.keys())[0]
file_content = uploaded[file_name]

In [6]:
import PyPDF2
import io


def extract_text_from_pdf_content(pdf_content):
    text = ""
    # Convert the file content into a file-like object
    pdf_file = io.BytesIO(pdf_content)
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    # Iterate through each page and extract text
    num_pages = len(pdf_reader.pages)
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

# Extract text from the uploaded PDF content
text = extract_text_from_pdf_content(file_content)
#print(text)

##CHUNK  
First we will try to chunk by paragraph

First we clean the data by removing every words in Upper case letter that are not title to allow us to split the doc in chunks.

In [24]:
text_clean = text.replace("VOUS ETES LIBERE DE PRISON", "Vous etes libere de prison") \
                 .replace("ALLEZ EN PRISON", "allez en prison") \
                 .replace("CHANCE", "Chance")\
                 .replace("IMPOTS","impots")\
                 .replace("GARE","gare")\
                 .replace("LE VAINQUEUR", "le vainqueur")\
                 .replace("PARC GRATUIT","parc gratuit")\
                 .replace("TAXE DE LUXE","taxe de luxe")\
                 .replace("COMPAGNIE DE DISTRIBUTION", "compagnie de distribution")\
                 .replace("COMMUNAUTE","communaute")\
                 .replace("VOUS ETES", "vous etes") \
                 .replace("LIBERE DE PRISON","libere de prison")\
                 .replace("FIN DE LA PARTIE","fin de la partie")


In [25]:
import re

# Split the clean data by paragraphes
paragraphs = re.split(r'\n(?=[A-Z]{5})', text_clean.strip(), flags=re.MULTILINE)

the split generate 5 paragraphes

In [26]:
len(paragraphs)

5

##Tokenise